Run SCENIC+ GRN Inference
1:

In [ ]:
# 1a) Prepare expression matrix for SCENIC+
ex_matrix = pd.DataFrame(adata.X.toarray(), 
                         index=adata.obs_names, 
                         columns=adata.var_names)

# 1b) Infer co-expression modules
network = sp.grnboost2(ex_matrix, 
                       tf_names=sp.default_tfs,  # default TF list
                       verbose=True)

2: Motif Pruning with RcisTarget

In [ ]:
# 2a) Load the hg38 motif rankings (downloaded ahead of time)
db_fnames = ["hg38__refseq-r80__10kb_upstream__500bp_downstream"]  

# 2b) Run motif enrichment to prune modules to regulons
regulons = sp.prune2df(network, ex_matrix, db_fnames, n_workers=4)

3: Score Regulon Activity (AUCell)

In [ ]:
# 3a) Build regulon–gene mapping
regulon_dict = sp.df2regulons(regulons, minsize=3)

# 3b) Compute AUCell scores per cell
auc_mtx = sp.score_regulons(ex_matrix, regulon_dict, n_workers=4)

# 3c) Store in AnnData
adata.obsm["X_regulonAUC"] = auc_mtx.loc[adata.obs_names].values

4: Explore Regulon Activities
4a. UMAP in regulon space

In [ ]:
sc.pp.neighbors(adata, use_rep="X_regulonAUC")
sc.tl.umap(adata)
sc.pl.umap(adata, color=["leiden"], title="Clusters vs. Regulon-UMAP")

4b. Heatmap of top regulons per cluster

In [ ]:
# Find top regulons driving each cluster
sc.pl.rank_genes_groups_heatmap(adata, groupby="leiden", use_rep="X_regulonAUC",
                               var_names=5, cmap="viridis", show_gene_labels=True)

5: Post-hoc Cluster Annotation
Now we’ll see which regulons (TF programs) are enriched in each of thw clusters. If we recognize a TF signature (e.g. MYC, TEAD1, HIF1A), we can assign a biological label to that cluster:

In [ ]:
# This is an example... check to see what is where before asigning these random ass labels. 
cluster_to_regulon = {
    "0": "MYC_high",
    "1": "Epithelial_HNF1B",
    "2": "Stromal_TEAD1",
    # … etc.
}

adata.obs["regulon_cluster"] = adata.obs["leiden"].map(cluster_to_regulon)
sc.pl.umap(adata, color="regulon_cluster", legend_loc="on data")
